In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd


from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D,GRU
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Concatenate
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow_addons as tfa
import json
import numpy as np
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [11]:
max_len=40
feat_vec = 300
with open ("../../encode_data_tiki.txt", "r") as f:
    encode_data_tiki = json.loads(f.read())
padded_doc_tiki=pad_sequences(encode_data_tiki,maxlen=max_len,padding="post")
with open ("../../output_tiki.txt", "r") as f:
    output_ori_tiki = np.array(json.loads(f.read()))
X_train_tiki, X_test_tiki, y_train_ori_tiki, y_test_ori_tiki = train_test_split(padded_doc_tiki, output_ori_tiki, test_size=0.2, random_state=14)


In [25]:
y_test_tiki = np.delete(y_test_ori_tiki, 6, 1)


In [12]:
embed_trained = np.load('./weights/embedding_trained_mat.npy')
lstm_trained = np.load('./weights/lstm_trained_mat.npy', allow_pickle=True)
# len(lstm_trained)
embed_mat = np.load('./weights/embedding_mat.npy')
# len(embed_mat)
len(embed_trained[0])

4514

In [66]:
f1 = tfa.metrics.F1Score(num_classes=6, average='macro', threshold=0.5)
def get_model():
    inp = Input(shape=(max_len,))
#     x = Embedding(len(embed_mat), feat_vec, weights=[embed_mat], input_length=max_len,trainable=True)(inp) 

    embed_distill = Embedding(len(embed_trained[0]), feat_vec, weights=[embed_trained[0]], input_length=max_len,trainable=False)(inp) # trọng số của từ sẽ không được train lại
    embed_train = Embedding(len(embed_trained[0]), feat_vec, weights=[embed_mat], input_length=max_len,trainable=True)(inp)
#     reduce_embed_train = Dense(270, activation="relu")(embed_train)

#     embed_concat = Concatenate(axis=-1)([0.4*embed_distill,0.8*embed_train])
#     x = Dropout(0.25)(embed_concat)

    embed_combine = 0.8*embed_distill + 0.2*embed_train
    x = Dropout(0.25)(embed_combine)

#     lstm_distill = Bidirectional(LSTM(128, return_sequences=True))(x)
#     reduce_lstm_distill = Dense(205, activation="relu")(lstm_distill)
#     lstm_train = Bidirectional(LSTM(128, return_sequences=True))(x)
#     lstm_concat = Concatenate(axis=-1)([reduce_lstm_distill,lstm_train])
#     lstm_combine = 0.5*lstm_distill+0.5*lstm_train
    x = Bidirectional(LSTM(128, return_sequences=True), trainable=True)(x)

#     convo_distill = Conv1D(64,3,activation="relu")(lstm_combine)    
#     convo_train = Conv1D(64,3,activation="relu")(lstm_combine)
#     convo_concat = Concatenate(axis=-1)([0.2*convo_distill,0.8*convo_train])
#     convo_combine = 0.3*convo_distill+0.7*convo_train
    x = Conv1D(64,3,activation="relu")(x)
    x = GlobalMaxPool1D()(x)
#     x = Dropout(0.4)(x)

    x = Dense(128, activation="relu")(x)
#     x = Dense(32, activation="relu")(x)
#     x = Dropout(0.4)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
#     model.layers[2].trainable = False
    #     model.layers[11].trainable = False
#     print(model.layers[6].trainable)
    print(model.summary())

    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.005), metrics= [f1])
    return model

In [78]:
def get_baseline_model():
    inp = Input(shape=(max_len,))
    x = Embedding(len(embed_mat), feat_vec, weights=[np.array(embed_mat)], input_length=max_len,trainable=True)(inp) # trọng số của từ sẽ không được train lại
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Conv1D(64,3,activation="relu")(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(256, activation="relu")(x)
#     convo1 = Conv1D(64,3,activation="relu")(x)
#     convo1_maxpool = GlobalMaxPool1D()(convo1)
#     convo2 = Conv1D(128,4,activation="relu")(x)
#     convo2_maxpool = GlobalMaxPool1D()(convo2)
#     convo3 = Conv1D(64,5,activation="relu")(x)
#     convo3_maxpool = GlobalMaxPool1D()(convo3)
#     concat1 = Concatenate()([convo1_maxpool, convo2_maxpool])
#     concat2 = Concatenate()([concat1, convo3_maxpool])
#     x = Dense(256, activation="relu")(concat2)
    
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    print(model.summary())

    return model
    

In [80]:
best_model = get_model()
best_model.load_weights('./weights/best models/embed 300 cong weight 0.9 distill dropout embed 0.25(82.24%).hdf5')


Model: "functional_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 40)]         0                                            
__________________________________________________________________________________________________
embedding_35 (Embedding)        (None, 40, 300)      1354200     input_21[0][0]                   
__________________________________________________________________________________________________
embedding_36 (Embedding)        (None, 40, 300)      1354200     input_21[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_Mul_34 (TensorFlowO [(None, 40, 300)]    0           embedding_35[0][0]               
______________________________________________________________________________________

In [68]:
from sklearn import metrics
def evaluate(X_test, y_test, model):    
    y_pre = model.predict(X_test)
    for thresh in np.arange(0.1,0.9,0.01):
        print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(y_test,(y_pre>thresh).astype(int), average='macro')))
    return y_pre
y_pre=evaluate(X_test_tiki, y_test_tiki, best_model)

threshold 0.10 f1 score:0.721
threshold 0.11 f1 score:0.734
threshold 0.12 f1 score:0.740
threshold 0.13 f1 score:0.747
threshold 0.14 f1 score:0.751
threshold 0.15 f1 score:0.758
threshold 0.16 f1 score:0.759
threshold 0.17 f1 score:0.766
threshold 0.18 f1 score:0.771
threshold 0.19 f1 score:0.771
threshold 0.20 f1 score:0.773
threshold 0.21 f1 score:0.773
threshold 0.22 f1 score:0.779
threshold 0.23 f1 score:0.780
threshold 0.24 f1 score:0.784
threshold 0.25 f1 score:0.786
threshold 0.26 f1 score:0.790
threshold 0.27 f1 score:0.792
threshold 0.28 f1 score:0.795
threshold 0.29 f1 score:0.795
threshold 0.30 f1 score:0.796
threshold 0.31 f1 score:0.800
threshold 0.32 f1 score:0.801
threshold 0.33 f1 score:0.804
threshold 0.34 f1 score:0.806
threshold 0.35 f1 score:0.809
threshold 0.36 f1 score:0.811
threshold 0.37 f1 score:0.815
threshold 0.38 f1 score:0.818
threshold 0.39 f1 score:0.811
threshold 0.40 f1 score:0.812
threshold 0.41 f1 score:0.807
threshold 0.42 f1 score:0.800
threshold 

In [77]:
thresh=0.38

y_pre_round = []
for i in range(len(y_pre)):
    y_pre_round.append(1*(y_pre[i] >= thresh))
    y_pre_round[i] = y_pre_round[i].tolist()
    
import sys
sys.path.append(r"C:\Users\acer\Documents\hoc tap\khóa luận\mebe_domain")
from modules.evaluate import cal_aspect_prf

X = cal_aspect_prf(y_test_ori_tiki.tolist(), y_pre_round, num_of_aspect=6, verbal=True)


p: [0.9459459459459459, 0.8805970149253731, 0.75, 0.7559523809523809, 0.7708333333333334, 0.7384615384615385]
r: [0.8860759493670886, 0.9147286821705426, 0.7272727272727273, 0.8819444444444444, 0.7254901960784313, 0.8571428571428571]
f1: [0.9150326797385621, 0.8973384030418251, 0.7384615384615384, 0.814102564102564, 0.7474747474747475, 0.793388429752066]
micro: (0.8292682926829268, 0.861850443599493, 0.8452454940957116)
macro: (0.8206657350314067, 0.8271023998666468, 0.8224819865638475)


In [85]:
baseline_model = get_baseline_model()
baseline_model.load_weights('./weights/best models/shopee_base.h5')

Model: "functional_41"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        [(None, 40)]              0         
_________________________________________________________________
embedding_39 (Embedding)     (None, 40, 300)           1354200   
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 40, 256)           439296    
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 38, 64)            49216     
_________________________________________________________________
global_max_pooling1d_20 (Glo (None, 64)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 256)               16640     
_________________________________________________________________
dense_41 (Dense)             (None, 6)               

In [87]:
type(baseline_model)

tensorflow.python.keras.engine.functional.Functional

In [88]:
y_pre_base=evaluate(X_test_tiki, y_test_tiki, baseline_model)

threshold 0.10 f1 score:0.690
threshold 0.11 f1 score:0.694
threshold 0.12 f1 score:0.694
threshold 0.13 f1 score:0.697
threshold 0.14 f1 score:0.693
threshold 0.15 f1 score:0.696
threshold 0.16 f1 score:0.699
threshold 0.17 f1 score:0.700
threshold 0.18 f1 score:0.701
threshold 0.19 f1 score:0.704
threshold 0.20 f1 score:0.705
threshold 0.21 f1 score:0.706
threshold 0.22 f1 score:0.705
threshold 0.23 f1 score:0.703
threshold 0.24 f1 score:0.705
threshold 0.25 f1 score:0.704
threshold 0.26 f1 score:0.705
threshold 0.27 f1 score:0.706
threshold 0.28 f1 score:0.707
threshold 0.29 f1 score:0.707
threshold 0.30 f1 score:0.705
threshold 0.31 f1 score:0.706
threshold 0.32 f1 score:0.707
threshold 0.33 f1 score:0.704
threshold 0.34 f1 score:0.704
threshold 0.35 f1 score:0.705
threshold 0.36 f1 score:0.705
threshold 0.37 f1 score:0.707
threshold 0.38 f1 score:0.708
threshold 0.39 f1 score:0.709
threshold 0.40 f1 score:0.709
threshold 0.41 f1 score:0.709
threshold 0.42 f1 score:0.712
threshold 

In [213]:
thresh=0.99

y_pre_base_round = []
for i in range(len(y_pre_base)):
    y_pre_base_round.append(1*(y_pre_base[i] >= thresh))
    y_pre_base_round[i] = y_pre_base_round[i].tolist()

In [214]:
y_pre_base_round[2]

[0, 0, 0, 0, 1, 0]

In [215]:
y_pre_round[2]

[0, 0, 0, 0, 1, 0]

In [216]:
final_pre = np.logical_or(y_pre_round, y_pre_base_round)

In [217]:
final_pre = (1*final_pre).tolist()

In [218]:
X = cal_aspect_prf(y_test_ori_tiki.tolist(), final_pre, num_of_aspect=6, verbal=True)


p: [0.9152542372881356, 0.8450704225352113, 0.7352941176470589, 0.7277777777777777, 0.7027027027027027, 0.676056338028169]
r: [0.9113924050632911, 0.9302325581395349, 0.7575757575757576, 0.9097222222222222, 0.7647058823529411, 0.8571428571428571]
f1: [0.9133192389006343, 0.8856088560885608, 0.746268656716418, 0.808641975308642, 0.7323943661971832, 0.7559055118110235]
micro: (0.7938844847112118, 0.8884664131812421, 0.8385167464114832)
macro: (0.7852198515901773, 0.8547257650707494, 0.8172466186422878)
